In [1]:
import sys
sys.path.append("..")

In [2]:
import re

from collections import Counter, defaultdict, namedtuple

from resources.utils import get_puzzle_input

### Part 1

You've sneaked into another supply closet - this time, it's across from the prototype suit manufacturing lab. You need to sneak inside and fix the issues with the suit, but there's a guard stationed outside the lab, so this is as close as you can safely get.

As you search the closet for anything that might help, you discover that you're not the first person to want to sneak in. Covering the walls, someone has spent an hour starting every midnight for the past few months secretly observing this guard post! They've been writing down the ID of the one guard on duty that night - the Elves seem to have decided that one guard was enough for the overnight shift - as well as when they fall asleep or wake up while at their post (your puzzle input).

For example, consider the following records, which have already been organized into chronological order:

```
[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
[1518-11-01 00:30] falls asleep
[1518-11-01 00:55] wakes up
[1518-11-01 23:58] Guard #99 begins shift
[1518-11-02 00:40] falls asleep
[1518-11-02 00:50] wakes up
[1518-11-03 00:05] Guard #10 begins shift
[1518-11-03 00:24] falls asleep
[1518-11-03 00:29] wakes up
[1518-11-04 00:02] Guard #99 begins shift
[1518-11-04 00:36] falls asleep
[1518-11-04 00:46] wakes up
[1518-11-05 00:03] Guard #99 begins shift
[1518-11-05 00:45] falls asleep
[1518-11-05 00:55] wakes up
```

Timestamps are written using year-month-day hour:minute format. The guard falling asleep or waking up is always the one whose shift most recently started. Because all asleep/awake times are during the midnight hour (00:00 - 00:59), only the minute portion (00 - 59) is relevant for those events.

Visually, these records show that the guards are asleep at these times:

```
Date   ID   Minute
            000000000011111111112222222222333333333344444444445555555555
            012345678901234567890123456789012345678901234567890123456789
11-01  #10  .....####################.....#########################.....
11-02  #99  ........................................##########..........
11-03  #10  ........................#####...............................
11-04  #99  ....................................##########..............
11-05  #99  .............................................##########.....
```

The columns are Date, which shows the month-day portion of the relevant day; ID, which shows the guard on duty that day; and Minute, which shows the minutes during which the guard was asleep within the midnight hour. (The Minute column's header shows the minute's ten's digit in the first row and the one's digit in the second row.) Awake is shown as ., and asleep is shown as #.

Note that guards count as asleep on the minute they fall asleep, and they count as awake on the minute they wake up. For example, because Guard #10 wakes up at 00:25 on 1518-11-01, minute 25 is marked as awake.

If you can figure out the guard most likely to be asleep at a specific time, you might be able to trick that guard into working tonight so you can have the best chance of sneaking in. You have two strategies for choosing the best guard/minute combination.

Strategy 1: Find the guard that has the most minutes asleep. What minute does that guard spend asleep the most?

In the example above, Guard #10 spent the most minutes asleep, a total of 50 minutes (20+25+5), while Guard #99 only slept for a total of 30 minutes (10+10+10). Guard #10 was asleep most during minute 24 (on two days, whereas any other minute the guard was asleep was only seen on one day).

While this example listed the entries in chronological order, your entries are in the order you found them. You'll need to organize them before they can be analyzed.

What is the ID of the guard you chose multiplied by the minute you chose? (In the above example, the answer would be 10 * 24 = 240.)

In [3]:
# [1518-11-01 00:00] Guard #10 begins shift
line_re = re.compile(r'^\[(....)-(..)-(..) (..):(..)\] (.+)$')
guard_re = re.compile(r'Guard #([0-9]+) begins shift')

In [4]:
assert line_re.match('[1518-11-01 00:00] Guard #10 begins shift').groups() == (
    '1518',
    '11',
    '01',
    '00',
    '00',
    'Guard #10 begins shift'
)
assert guard_re.match('Guard #10 begins shift').groups()[0] == '10'

In [5]:
GuardState = namedtuple('GuardState', [
    'guard',
    'asleep',
    'year',
    'month',
    'day',
    'hour',
    'minute'
])

In [6]:
def parse_line(line):
    match = line_re.match(line)

    if not match:
        raise ValueError('Cannot parse {}'.format(line))

    year, month, day, hour, minute = (int(i) for i in match.groups()[:5])
    text = match.groups()[5]
    
    if text == 'falls asleep':
        guard = None
        asleep = True
    elif text == 'wakes up':
        guard = None
        asleep = False
    else:
        match = guard_re.match(text)
        if not match:
            raise ValueError('Cannot parse text {}'.format(text))
        
        guard = match.groups()[0]
        asleep = False
    
    return GuardState(
        guard=guard,
        asleep=asleep,
        year=year,
        month=month,
        day=day,
        hour=hour,
        minute=minute
    )

In [7]:
parse_line('[1518-11-01 00:00] Guard #10 begins shift')

GuardState(guard='10', asleep=False, year=1518, month=11, day=1, hour=0, minute=0)

In [8]:
parse_line('[1518-11-02 00:40] falls asleep')

GuardState(guard=None, asleep=True, year=1518, month=11, day=2, hour=0, minute=40)

In [9]:
parse_line('[1518-11-04 00:46] wakes up')

GuardState(guard=None, asleep=False, year=1518, month=11, day=4, hour=0, minute=46)

In [10]:
def sleepiest_guard(sorted_input):
    guard = None
    minute_asleep = 0
    asleep = False
    guard_time_asleep = Counter()
    guard_minute_asleep = defaultdict(Counter)
    
    for line in sorted_input:
        state = parse_line(line)
        if state.guard:
            assert not asleep
            guard = state.guard
            continue
        
        if state.asleep:
            #Sleeping
            assert guard and not asleep
            minute_asleep = state.minute
            asleep = True
        else:
            # Waking up
            assert asleep
            minutes_asleep = list(range(minute_asleep, state.minute))
            guard_time_asleep[guard] += len(minutes_asleep)
            
            for minute in minutes_asleep:
                guard_minute_asleep[guard][minute] += 1
            
            asleep = False
    
    sleepiest_guard, _ = guard_time_asleep.most_common()[0]
    most_common_minute, _ = guard_minute_asleep[sleepiest_guard].most_common()[0]
    
    # Part 2: Look at the guard that was asleep in the same minute
    # for the longest time
    max_time = 0
    longest_in_minute = None

    for guard in guard_minute_asleep:
        common_minute, time_asleep = guard_minute_asleep[guard].most_common()[0]
        
        if time_asleep > max_time:
            longest_in_minute = guard, common_minute
            max_time = time_asleep
    
    return (sleepiest_guard, most_common_minute), longest_in_minute 
   

In [11]:
test_input = [
    '[1518-11-01 00:00] Guard #10 begins shift',
    '[1518-11-01 00:05] falls asleep',
    '[1518-11-01 00:25] wakes up',
    '[1518-11-01 00:30] falls asleep',
    '[1518-11-01 00:55] wakes up',
    '[1518-11-01 23:58] Guard #99 begins shift',
    '[1518-11-02 00:40] falls asleep',
    '[1518-11-02 00:50] wakes up',
    '[1518-11-03 00:05] Guard #10 begins shift',
    '[1518-11-03 00:24] falls asleep',
    '[1518-11-03 00:29] wakes up',
    '[1518-11-04 00:02] Guard #99 begins shift',
    '[1518-11-04 00:36] falls asleep',
    '[1518-11-04 00:46] wakes up',
    '[1518-11-05 00:03] Guard #99 begins shift',
    '[1518-11-05 00:45] falls asleep',
    '[1518-11-05 00:55] wakes up'
]

In [13]:
assert sleepiest_guard(test_input) == (('10', 24), ('99', 45))

In [14]:
puzzle_input = get_puzzle_input('/tmp/day4.txt')

In [15]:
sleepiest_guard(sorted(puzzle_input))

(('499', 44), ('3449', 39))

In [16]:
499 * 44

21956

In [17]:
3449 * 39

134511

### Part 2

Strategy 2: Of all guards, which guard is most frequently asleep on the same minute?

In the example above, Guard #99 spent minute 45 asleep more than any other guard or minute - three times in total. (In all other cases, any guard spent any minute asleep at most twice.)

What is the ID of the guard you chose multiplied by the minute you chose? (In the above example, the answer would be 99 * 45 = 4455.)